In [201]:
import numpy as np
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-01-17 14:35:37--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.30MB/s    in 0.3s    

2025-01-17 14:35:38 (3.30 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [228]:
from pathlib import Path

In [229]:
with Path("input.txt").open("r", encoding="utf-8") as f:
    text = f.read()

In [272]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab = ''.join(chars)
# print(vocab_size)
# print(vocab)

In [273]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [274]:
# take a string and output a list of ints
encode = lambda s: [stoi[c] for c in s]
decode = lambda e: "".join([itos[i] for i in e])
# print(encode("hii there"))
# print(decode(encode("hii there")))

In [275]:
# encode(text)

In [276]:
data = torch.tensor(encode(text), dtype=torch.long)
# print(data.shape, data.dtype)
# print(data[:1000])

In [277]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [311]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200
n_embed = 32
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [312]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [314]:
# torch.randint(len(train_data) - block_size, (batch_size, ))
xb, yb = get_batch("train")
# xb, yb
print(xb)
# print(xb.shape)
# print(yb)
# print(yb.shape)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [288]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # Init at random a learnable embedding matrix
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        # Each position from 0 to block_size - 1 will
        # get its own positional embedding added on as an offset
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # pos_emb is broadcasted across each b in B for tok_emb
        logits = self.lm_head(x) # (B, T, vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            # print(i)
            # Get the prediction for the next token given the previous token(s)
            logits, _ = self(idx)
            # Select the predictions only
            # print('before')
            # print(logits.shape)
            # print(logits)
            logits = logits[:, -1, :] # (B, C)
            # print('after')
            # print(logits.shape)
            # print(logits)
            # Run the logits (value of embedding vector weights) through
            # softmax to get probability distribution
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution to get an index of a vector (a character)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Concatenate the idx to the current idx to create the next
            # idx vector to pass into `self` (forward pass function)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            # print()
        return idx

In [289]:
model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
# print(logits[:, -1])

In [290]:
# emb will lookup and return a tensor of shape (m, vocab_size) or ((B, T), vocab_size)
# of the vectors corresponding to the indices of the tensor you pass in
# val = emb(torch.tensor((5, 39, 2, 58, 29)))
# print(val.shape)

In [293]:
in_ = torch.zeros((1, 1), dtype=torch.long)
print(in_)
out = m.generate(in_, max_new_tokens=100)
# print(out)
print(decode(out[0].tolist()))

tensor([[0]])

jtbsA B$-f3jIZZ
 &$NK3OixTFP&xyAuYa:BKNTFof?Mdr3CZcWac:y$::XqM!$nCyEavmFzqc-HW' 


lwjNG
dQmA$SHQmrc


In [313]:
@torch.no_grad() # we don't intend to do backprop so don't store all intermediate variables
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [294]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [295]:
batch_size = 32
for iter_ in range(max_iters):
    if iter_ % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter_} train_loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # print(loss.item())
print(loss.item())

step 0 train_loss 4.3352, val loss 4.3310
step 200 train_loss 3.1085, val loss 3.1087
step 400 train_loss 2.7614, val loss 2.7712
step 600 train_loss 2.6524, val loss 2.6645
step 800 train_loss 2.5840, val loss 2.6198
step 1000 train_loss 2.5635, val loss 2.5826
step 1200 train_loss 2.5375, val loss 2.5584
step 1400 train_loss 2.5194, val loss 2.5520
step 1600 train_loss 2.5115, val loss 2.5386
step 1800 train_loss 2.5146, val loss 2.5312
step 2000 train_loss 2.5170, val loss 2.5217
step 2200 train_loss 2.5049, val loss 2.5217
step 2400 train_loss 2.4941, val loss 2.5121
step 2600 train_loss 2.4959, val loss 2.5175
step 2800 train_loss 2.4854, val loss 2.5190
2.493168592453003


In [296]:
in_ = torch.zeros((1, 1), dtype=torch.long, device=device)
print(in_)
out = m.generate(in_, max_new_tokens=100)
# print(out)
print(decode(out[0].tolist()))

tensor([[0]])

NTh haveo and ink! I sul Bed hernongr;
REO: wil,
Wissome garesth oust he ovee perrear hanorerin gew,


In [317]:
# import v3
# v3.main()

tensor([[0]])

OOPXyH.!c.qpQPbhhy,OdB!g elctWZW?Otd?dtwpRCgbGmY.-uB'Kmg$noIwEM:V3pWg;:d b-MPTW-Jf!
$fLj$gOPtqJbABqd
step 0 train_loss 4.2017, val loss 4.1999
step 200 train_loss 3.0297, val loss 3.0607
step 400 train_loss 2.8234, val loss 2.8414
step 600 train_loss 2.6575, val loss 2.6586
step 800 train_loss 2.5733, val loss 2.5723
step 1000 train_loss 2.5129, val loss 2.5311
step 1200 train_loss 2.4840, val loss 2.5102
step 1400 train_loss 2.4692, val loss 2.4873
step 1600 train_loss 2.4561, val loss 2.4720
step 1800 train_loss 2.4392, val loss 2.4778
step 2000 train_loss 2.4293, val loss 2.4659
step 2200 train_loss 2.4416, val loss 2.4519
step 2400 train_loss 2.4335, val loss 2.4391
step 2600 train_loss 2.4209, val loss 2.4448
step 2800 train_loss 2.4184, val loss 2.4427



KeyboardInterrupt

